# Efficient Global Optimization employing Deep Gaussian Process Regression

See DGPR implementations in:

https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html
https://docs.gpytorch.ai/en/stable/examples/06_PyTorch_NN_Integration_DKL/KISSGP_Deep_Kernel_Regression_CUDA.html

In [ ]:
from ansys.mapdl.core import launch_mapdl
import pandas as pd

def launch_mapdl_on_available_port(starting_port=50052, max_attempts=5):
    for i in range(max_attempts):
        port = starting_port + i
        try:
            mapdl = launch_mapdl(port=port)
            print(f"MAPDL launched successfully on port {port}")
            return mapdl
        except Exception as e:
            print(f"Failed to launch MAPDL on port {port}: {e}")
    raise RuntimeError("Could not launch MAPDL on any available port")

# Use a função para iniciar uma instância do MAPDL
mapdl = launch_mapdl_on_available_port()

In [ ]:
import math
import numpy as np  # Para usar np.round

# Definindo variáveis apenas uma vez
raio_estaca = 0.5
distancia_minima = raio_estaca * 2.5  # 5 vezes o raio da estaca
largura = 6
altura = 3
n_estacas = 2
n_estacas_centrais = round(1) 

def round_to_nearest(value, targets=[0, 0.333, 0.666, 1]):
    return min(targets, key=lambda t: abs(value - t))

def distancia_entre_pontos(p1, p2):
    return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

def espelhar_pontos(coordinates, n_estacas_centrais):
    espelho = [(x, -y, z) for x, y, z in coordinates[:-int(n_estacas_centrais)]]
    return espelho

def converter_x(x_normalizado):
    return 0.5 + x_normalizado * (5.5 - 0.5)

def converter_y(y_normalizado):
    return 0.5 + y_normalizado * (2.5 - 0.5)

def converter_angulos_verticais(array_angulos_verticais):
    return [np.round(75 + 15 * ang_v, 2) for ang_v in array_angulos_verticais]

def converter_angulos_horizontais(array_angulos_horizontais):
    return [np.round(46 + 268 * ang_h, 2) for ang_h in array_angulos_horizontais]

def spherical_to_cartesian(x_inicial, y_inicial, z_inicial, L_estacas, angulos_verticais_theta, angulos_horizontais_theta):
    angulo_vertical_rad = np.deg2rad(angulos_verticais_theta)
    angulo_horizontal_rad = np.deg2rad(angulos_horizontais_theta)
    
    componente_x = L_estacas * np.cos(angulo_vertical_rad) * np.cos(angulo_horizontal_rad)
    componente_y = L_estacas * np.cos(angulo_vertical_rad) * np.sin(angulo_horizontal_rad)
    componente_z = L_estacas * np.sin(angulo_vertical_rad)
    
    x_final = x_inicial + componente_x
    y_final = y_inicial + componente_y
    z_final = z_inicial + componente_z

    return x_final, y_final, z_final

def gerar_pontos_finais(pontos, angulos_verticais_theta, angulos_horizontais_theta):
    coordenadas_finais = []
    for i in range(len(pontos)):
        x, y, z = pontos[i]
        theta_v, theta_h = angulos_verticais_theta[i], angulos_horizontais_theta[i]
        x_final, y_final, z_final = spherical_to_cartesian(x, y, z, L_estacas, theta_v, theta_h)
        coordenadas_finais.append((round(x_final, 2), round(y_final, 2), -1*round(z_final, 2)))
    return coordenadas_finais

def espelhar_pontos_finais(pontos, k):
    espelho = []
    for (x, y, z) in pontos[:-k]:
        y_espelhado = -y
        espelho.append((x, y_espelhado, z))
    return espelho

def distancia_entre_pontos_r3(p1, p2):
    return round(np.linalg.norm(p1 - p2), 2)

def encontrar_menor_distancia_entre_vetores(pontos_iniciais, pontos_finais, k, l, comprimento):
    vetor_k = pontos_finais[k] - pontos_iniciais[k]
    vetor_l = pontos_finais[l] - pontos_iniciais[l]

    vetor_dividido_k = vetor_k / comprimento
    vetor_dividido_l = vetor_l / comprimento

    menor_distancia = float('inf')
    ponto_menor_distancia_v1 = None
    ponto_menor_distancia_v2 = None
    N_v1 = None
    N_v2 = None

    for N1 in np.arange(1, 19, 1):
        for N2 in np.arange(1, 19, 1):
            ponto_v1 = pontos_iniciais[k] + N1 * vetor_dividido_k
            ponto_v2 = pontos_iniciais[l] + N2 * vetor_dividido_l

            dist = round(distancia_entre_pontos_r3(ponto_v1, ponto_v2), 2)

            if dist < menor_distancia:
                menor_distancia = dist
                ponto_menor_distancia_v1 = ponto_v1
                ponto_menor_distancia_v2 = ponto_v2
                N_v1 = N1
                N_v2 = N2

    return menor_distancia, ponto_menor_distancia_v1, ponto_menor_distancia_v2, N_v1, N_v2

def gerar_lista_k(array_angulos_verticais, n_estacas, n_estacas_centrais):
    indices = list(range(len(array_angulos_verticais)))
    indices_menores = sorted(range(len(array_angulos_verticais) - 1), key=lambda i: array_angulos_verticais[i])[:n_estacas]
    lista_k = [i for i in indices_menores]
    lista_k.append(2)
    if len(lista_k) < n_estacas + n_estacas_centrais and lista_k.count(2) < 2:
        lista_k.append(2)
    return lista_k

def verificar_e_ajustar_pontos(pontos, L_estacas, distancia_minima, pontos_finais, array_angulos_verticais_convertidos, array_angulos_horizontais_convertidos, array_angulos_horizontais,p):
    max_iteracoes=1*(n_estacas+n_estacas_centrais+n_estacas)
    iteracao = 0
    pontos_finais = np.array(pontos_finais)
    incremento = 0.1
    print("P atual1",p)
    lista_k = gerar_lista_k(array_angulos_verticais_convertidos, n_estacas, n_estacas_centrais)
    trial=5

    for k in lista_k:
        while iteracao <= max_iteracoes:
            ajustou_ponto = False

            for l in range(len(pontos_finais)):
                if k == l:
                    continue
                ponto1 = pontos_finais[k]
                ponto2 = pontos_finais[l]
                dist = distancia_entre_pontos_r3(ponto1, ponto2)
                menor_distancia, _, _, _, _ = encontrar_menor_distancia_entre_vetores(pontos, pontos_finais, k, l, L_estacas)
                if menor_distancia < 3 * raio_estaca or dist < 5 * raio_estaca:
                    ajustou_ponto = True
                    if k <= n_estacas:
                        array_angulos_horizontais_convertidos[k] = array_angulos_horizontais_convertidos[k]+incremento
                        array_angulos_horizontais[k] = array_angulos_horizontais[k]+incremento
                    else:
                        array_angulos_horizontais_convertidos[k] = array_angulos_horizontais_convertidos[k]+incremento
                        array_angulos_horizontais[k] = array_angulos_horizontais[k]+incremento

                    x_novo, y_novo, z_novo = spherical_to_cartesian(
                        pontos[k][0], pontos[k][1], pontos[k][2],
                        L_estacas, array_angulos_verticais_convertidos[k], array_angulos_horizontais_convertidos[k]
                    )
                    x = np.round(x_novo, 2)
                    y = np.round(y_novo, 2)
                    z = -1*np.round(z_novo, 2)

                    if k < n_estacas:
                        pontos_finais[k, :] = [x, y, z]
                        pontos_finais[k + n_estacas + n_estacas_centrais, :] = [x, -y, z]
                    else:
                        pontos_finais[k, :] = [x, y, z]
                    break

            if not ajustou_ponto:
                break
            trial += 1
            
            if trial >= max_iteracoes:
                print("P atual",p)
                print("k atual",k)
                print("i atual",iteracao)
                p+=6
                k+=1
                iteracao+=1

                break

    return pontos_finais, array_angulos_horizontais_convertidos, array_angulos_horizontais,p


def analise(pontos_iniciais, pontos_finais_ajustados, array_angulos_horizontais, array_angulos_verticais):
    mapdl.clear('NOSTART')
    mapdl.prep7()

    # Título
    mapdl.title('Análise de Estacas e Casca')

    # Definir o tipo de elemento (BEAM188) e suas propriedades
    mapdl.et(1, 'BEAM188')

    # Propriedades do material
    modulo_elasticidade = 0.85 * 5600 * (40 ** 0.5) * 1e6  # N/m²
    mapdl.mp('EX', 1, modulo_elasticidade)
    mapdl.mp('PRXY', 1, 0.2)  # Coeficiente de Poisson
    mapdl.mp('DENS', 1, 2500)  # Densidade

    # Propriedades da seção da viga
    mapdl.sectype(1, 'BEAM', 'CSOLID')
    mapdl.secoffset('CENT')
    mapdl.secdata(0.5)

    # Número de nós intermediários
    num_intermediate_nodes = 9

    # Adicionar nós
    node_id = 1
    for i in range(2 * n_estacas + n_estacas_centrais):
        # Obter coordenadas iniciais e finais
        x_inicial, y_inicial, z_inicial = pontos_iniciais[i]
        x_final, y_final, z_final = pontos_finais_ajustados[i]

        # Criar o nó inicial
        mapdl.n(node_id, x_inicial, y_inicial, z_inicial)

        # Definir o ID do nó final
        node_id_final = node_id + num_intermediate_nodes + 1

        # Criar o nó final
        mapdl.n(node_id_final, x_final, y_final, z_final)

        # Travar o nó final
        mapdl.d(node_id_final, 'ALL', 0)

        # Preencher nós intermediários
        mapdl.fill(node_id, node_id_final, num_intermediate_nodes)

        # Atualizar o próximo node_id
        node_id = node_id_final + 1

    # Geração de elementos
    n_elemento = 1
    for i in range(2 * n_estacas + n_estacas_centrais):
        for j in range(1, num_intermediate_nodes + 2):
            N_1 = j + (num_intermediate_nodes + 2) * i
            N_2 = N_1 + 1
            mapdl.en(n_elemento, N_1, N_2)
            n_elemento += 1

    # Selecionar elementos tipo BEAM188
    mapdl.esel('S', 'TYPE', '', 1)

    # Contar elementos selecionados
    num_elem = mapdl.get('num_elem', 'ELEM', 0, 'COUNT')
    k = num_elem

    # Criar elemento de carga
    mapdl.n(1000, 3, 0, 0)
    mapdl.n(1001, 3, 0, 0.1)
    mapdl.en(k + 1, 1000, 1001)

    # Definir tipo de elemento SHELL181
    mapdl.et(2, 'SHELL181')
    mapdl.keyopt(2, 8, 2)  # Elastoplástico
    mapdl.keyopt(2, 3, 2)  # Precisão de tensões

    # Propriedades do material para SHELL181
    modulo_elasticidade_shell = 0.85 * 5600 * (20 ** 0.5) * 1e6
    mapdl.mp('EX', 2, modulo_elasticidade_shell)
    mapdl.mp('PRXY', 2, 0.2)
    mapdl.mp('DENS', 2, 2500)

    # Definir seção de casca
    mapdl.sectype(2, 'SHELL')
    mapdl.secdata(1.5)

    # Criar retângulo e malhar
    mapdl.rectng(0, 6, -3, 3)
    mapdl.esize(0.1)
    mapdl.amesh('ALL')

    # Selecionar elementos tipo SHELL181
    mapdl.esel('S', 'TYPE', '', 2)
    mapdl.emodif('ALL', 'SECNUM', 2)

    # Merge de nós
    mapdl.nsel('S', 'LOC', 'Z', 0, 1e5)
    mapdl.nummrg('NODE', 1e-5)
    mapdl.nsel('ALL')

    # Saída dos resultados
    mapdl.allsel('ALL')

    # Aplicar gravidade
    mapdl.acel(0, 0, -9.81)

    # Finalizar
    mapdl.finish()

    # Entrar no modo de solução
    mapdl.slashsolu()
    mapdl.antype(0)

    import math

    # Definir a força aplicada e o número de load steps
    f = 1000000  # Força de 1 milhão de N
    casos = 12  # Definir loadsteps

    # Loop para aplicar as diferentes condições de carga (12 casos de 15 em 15 graus)
    for i in range(1, casos + 2):
        Rad = (i - 1) * (180 / casos) * math.pi / 180  # Variação de 15 graus

        # Calcular as componentes da força horizontal
        FX = math.cos(Rad) * f
        FY = math.sin(Rad) * f

        # Selecionar todos os elementos
        mapdl.allsel('ALL')

        # Aplicar as forças horizontais no nó 1001
        mapdl.f(1001, 'FX', FX)
        mapdl.f(1001, 'FY', FY)

        # Resolver o modelo
        mapdl.solve()

        # Salvar os resultados da solução
        mapdl.save(f'load_step_{i}')

    # Finalizar a análise
    mapdl.finish()

    # Definir o número de casos
    casos = 12

    # Entrar no modo de pós-processamento
    mapdl.post1()

    # Definir os Load Cases para N = 12
    for i in range(1, casos + 2):
        mapdl.lcdef(i, i, 1)  # Define o Load Case i para o Load Step i

    # Carregar o primeiro Load Case
    mapdl.lcase(1)

    #!!!!!!!!!!!!!!!!!!!!!CÁLCULO DO MAX!!!!!!!!!!!!!!!!!!!!!
    # Comparar o Load Case 1 com os demais e armazenar os resultados (máximos)
    for R in range(2, casos + 2):
        mapdl.lcoper('MAX', R)  # Compara o Load Case na memória com os próximos
        mapdl.lcwrite(50 + R)   # Escreve o resultado em um arquivo

    mapdl.lcase(50 + (casos + 1))  # Carregar o último load case comparado
    mapdl.etable('Fx_MAX', 'SMISC', 1, 'MAX')  # Força Axial máxima

    #!!!!!!!!!!!!!!!!!!!!!CÁLCULO DO MIN!!!!!!!!!!!!!!!!!!!!!
    # Carregar o primeiro Load Case novamente
    mapdl.lcase(1)

    # Comparar o Load Case 1 com os demais para valores mínimos
    for R in range(2, casos + 2):
        mapdl.lcoper('MIN', R)  # Compara o Load Case na memória para valores mínimos
        mapdl.lcwrite(60 + R)   # Escreve o resultado em um arquivo

    mapdl.lcase(60 + (casos + 1))  # Carregar o último load case comparado
    mapdl.etable('Fx_MIN', 'SMISC', 1, 'MIN')  # Força Axial mínima

    #!!!!!!!!!!!!!!!!!!!!!CÁLCULO DO ABSMAX!!!!!!!!!!!!!!!!!!!!!
    # Carregar o primeiro Load Case
    mapdl.lcase(1)

    # Comparar o Load Case 1 com os demais e armazenar os resultados (máximos absolutos)
    for R in range(2, casos + 2):
        mapdl.lcoper('ABMX', R)  # Compara o Load Case na memória com os próximos
        mapdl.lcwrite(50 + R)   # Escreve o resultado em um arquivo

    mapdl.lcase(50 + (casos + 1))  # Carregar o último load case comparado
    mapdl.etable('My_ABMX', 'SMISC', 2, 'ABMX')  # Momento fletor máximo em Y
    mapdl.etable('Mz_ABMX', 'SMISC', 3, 'ABMX')  # Momento fletor máximo em Z

    # Gerar ETABLE para deslocamentos dos nós selecionados
    mapdl.etable('UX', 'U', 'X')  # Deslocamento em X
    mapdl.etable('UY', 'U', 'Y')  # Deslocamento em Y
    mapdl.etable('UZ', 'U', 'Z')  # Deslocamento em Z

    mapdl.eplot()

    import pandas as pd
    import math

    # Definir a sequência de elementos
    elementos_sequencia = []
    j = 1
    for i in range(0, (2 * n_estacas) + n_estacas_centrais, 1):
        y = num_intermediate_nodes * i + j
        elementos_sequencia.append(y)
        j += 1

    # Dicionário para armazenar os resultados
    data = {
        "Fx_MAX": [],
        "Fx_MIN": [],
        "Utot": [],
        "My": [],
        "Mz": [],
        "r_max": [],
        "load_case_max": []
    }

    # Primeiro loop: Calculando Fx_MAX, Fx_MIN e Utot para cada elemento
    for elem in elementos_sequencia:
        fx_max = mapdl.get_value('ELEM', elem, 'ETABLE', 'Fx_MAX') / 1000  # kN
        fx_min = mapdl.get_value('ELEM', elem, 'ETABLE', 'Fx_MIN') / 1000  # kN
        ux = mapdl.get_value('ELEM', elem, 'ETABLE', 'UX') * 1000  # mm
        uy = mapdl.get_value('ELEM', elem, 'ETABLE', 'UY') * 1000  # mm
        uz = mapdl.get_value('ELEM', elem, 'ETABLE', 'UZ') * 1000  # mm
        Utot = math.sqrt(ux**2 + uy**2 + uz**2)

        # Adicionando os valores calculados ao dicionário
        data["Fx_MAX"].append(fx_max)
        data["Fx_MIN"].append(fx_min)
        data["Utot"].append(Utot)

    # Segundo loop: Calculando r_max, My, e Mz para cada elemento em todos os load cases
    for elem in elementos_sequencia:
        r_max_elem = 0
        my_max_elem = 0
        mz_max_elem = 0
        load_case_max = 0

        for load_case in range(1, casos + 2):
            mapdl.lcase(load_case)

            mapdl.etable('My', 'SMISC', 2)
            mapdl.etable('Mz', 'SMISC', 3)

            my_val = mapdl.get_value('ELEM', elem, 'ETABLE', 'My') / 1000
            mz_val = mapdl.get_value('ELEM', elem, 'ETABLE', 'Mz') / 1000

            r_val = math.sqrt(my_val**2 + mz_val**2)

            if r_val > r_max_elem:
                r_max_elem = r_val
                my_max_elem = my_val
                mz_max_elem = mz_val
                load_case_max = load_case

        data["My"].append(my_max_elem)
        data["Mz"].append(mz_max_elem)
        data["r_max"].append(r_max_elem)
        data["load_case_max"].append(load_case_max)

    df = pd.DataFrame(data)
    df = df.round(1)

    def f(x):
        if x <= 0:
            return 0
        elif x > 0:
            return 6#(10**10) * np.exp(x)
        else:
            return np.nan

    delta1 = 5500
    delta2 = 2000
    delta3 = 2500
    delta4 = 125

    def calcular_Pt(df):
        df['Pt_Fx_MAX'] = 0.0
        df['Pt_Fx_MIN'] = 0.0
        df['Pt_r_max'] = 0.0
        df['Pt_Utot'] = 0.0

        p_total = 331350 #PESO DA ESTRUTURA

        for index, row in df.iterrows():
            if row['Fx_MAX'] > 0:
                df.at[index, 'Pt_Fx_MAX'] = f((abs(row['Fx_MAX']) - delta1) / delta1)
            else:
                df.at[index, 'Pt_Fx_MAX'] = f((abs(row['Fx_MAX']) - delta2) / delta2)

            if row['Fx_MIN'] > 0:
                df.at[index, 'Pt_Fx_MIN'] = f(((abs(row['Fx_MIN']) - delta1) / delta1))
            else:
                df.at[index, 'Pt_Fx_MIN'] = f(((abs(row['Fx_MIN']) - delta2) / delta2))

            df.at[index, 'Pt_r_max'] = f(((abs(row['r_max']) - delta3) / delta3))
            df.at[index, 'Pt_Utot'] = f(((abs(row['Utot']) - delta4) / delta4))

            p_total += (df.at[index, 'Pt_Fx_MAX'] + df.at[index, 'Pt_Fx_MIN'] +
                        df.at[index, 'Pt_r_max'] + df.at[index, 'Pt_Utot'])

        return float(p_total.round(1))

    p_total = calcular_Pt(df)
    return p_total, pontos_iniciais, pontos_finais_ajustados, array_angulos_horizontais, array_angulos_verticais


def objective_function(x, distancia_minima, n_estacas_centrais):
    p = 0

    x_aba_1, y_aba_1 = round(x[0], 1), round(x[1], 1)
    x_aba_2, y_aba_2 = round(x[4], 1), round(x[5], 1)
    x_central_1 = round(x[8], 1)

    x_aba_1, y_aba_1 = converter_x(x_aba_1), converter_y(y_aba_1)
    x_aba_2, y_aba_2 = converter_x(x_aba_2), converter_y(y_aba_2)
    x_central_1 = converter_x(x_central_1)

    coordinates = [
        [x_aba_1, y_aba_1, 0],
        [x_aba_2, y_aba_2, 0],
        [x_central_1, 0.0, 0.0]
    ]

    ang_v1, ang_h1 = round_to_nearest(x[2]), round_to_nearest(x[3])
    ang_v2, ang_h2 = round_to_nearest(x[6]), round_to_nearest(x[7])
    ang_v3, ang_h3 = round_to_nearest(x[9]), round_to_nearest(x[10])

    array_angulos_verticais = [ang_v1, ang_v2, ang_v3]
    array_angulos_horizontais = [ang_h1, ang_h2, ang_h3]

    array_angulos_verticais_convertidos = converter_angulos_verticais(array_angulos_verticais)
    array_angulos_horizontais_convertidos = converter_angulos_horizontais(array_angulos_horizontais)
    print("array_angulos_horizontais_convertidos ORIGINAIS",array_angulos_horizontais_convertidos)

    espelho = espelhar_pontos(coordinates, n_estacas_centrais)

    for i in range(len(coordinates)):
        for j in range(i + 1, len(coordinates)):
            if distancia_entre_pontos(coordinates[i], coordinates[j]) < distancia_minima:
                p += 6
            

    coordenadas_finais = gerar_pontos_finais(coordinates, array_angulos_verticais_convertidos, array_angulos_horizontais_convertidos)
    espelhos_finais = espelhar_pontos_finais(coordenadas_finais, int(n_estacas_centrais))

    pontos_iniciais = coordinates + espelho
    pontos_finais = np.array(coordenadas_finais + espelhos_finais)

    pontos_finais_ajustados, array_angulos_horizontais_convertidos, array_angulos_horizontais,p = verificar_e_ajustar_pontos(
    pontos_iniciais, L_estacas, distancia_minima, pontos_finais, array_angulos_verticais_convertidos, array_angulos_horizontais_convertidos, array_angulos_horizontais,p)

    p_obj, pontos_iniciais, pontos_finais_ajustados, array_angulos_horizontais, array_angulos_verticais = analise(
    pontos_iniciais,
    pontos_finais_ajustados,
    array_angulos_horizontais,
    array_angulos_verticais)

    p=p+p_obj


    return pontos_iniciais, pontos_finais_ajustados, array_angulos_verticais_convertidos, array_angulos_horizontais_convertidos, p

# Exemplo de uso
x = [0.0, 1, 1.000, 0, 0.0, 0.555555555, 1, 0.49, 0.5, 1, 0.70490718]

pontos_iniciais, pontos_finais, array_angulos_verticais, array_angulos_horizontais, p = objective_function(x, distancia_minima, n_estacas_centrais)

resultado = {
    "Pontos Iniciais": pontos_iniciais,
    "Pontos Finais": pontos_finais,
    "Angulos Verticais Convertidos": array_angulos_verticais,
    "Angulos Horizontais Convertidos": array_angulos_horizontais,
    "p": p
}

if p > 0:
    resultado['Penalidade'] = "Penalidade aplicada."
else:
    resultado['Penalidade'] = "Nenhuma penalidade aplicada."

resultado


In [4]:
import numpy as np

from scipy.stats import qmc
from scipy.special import erf

import torch
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.utils.grid import ScaleToBounds
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.settings import use_toeplitz, fast_pred_var
from torch.optim import Adam

from BSA import bsa

import matplotlib.pyplot as plt

In [ ]:
import numpy as np

from scipy.stats import qmc
from scipy.special import erf

import torch
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.utils.grid import ScaleToBounds
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.settings import use_toeplitz, fast_pred_var
from torch.optim import Adam

from BSA import bsa

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [ ]:
import numpy as np

from scipy.stats import qmc
from scipy.special import erf

import torch
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ExactGP
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, MaternKernel
from gpytorch.utils.grid import ScaleToBounds
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.settings import use_toeplitz, fast_pred_var
from torch.optim import Adam

from BSA import bsa

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

## *GPR Model*

### Gaussian Process Regression

In [ ]:
class GPRegressionModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(MaternKernel(nu=1.5))
        self.scale_to_bounds = ScaleToBounds(-1., 1.)
        
        # Store train_y for later use
        self.train_outputs = train_y

    def forward(self, x):
        projected_x = self.scale_to_bounds(x)
        mean_x = self.mean_module(projected_x)
        covar_x = self.covar_module(projected_x)
        return MultivariateNormal(mean_x, covar_x)


### Train model

In [ ]:
def train_model(x, g, training_iterations):
    # Initialize the models and likelihood
    likelihood = GaussianLikelihood()
    model = GPRegressionModel(train_x=x, train_y=g, likelihood=likelihood)

    # if torch.cuda.is_available():
    #     model = model.cuda()
    #     likelihood = likelihood.cuda()

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = Adam([
        {'params': model.covar_module.parameters()},
        {'params': model.mean_module.parameters()},
        {'params': model.likelihood.parameters()},
    ], lr=0.01)

    # "Loss" for GPs - the marginal log likelihood
    mll = ExactMarginalLogLikelihood(likelihood, model)

    # Example training loop
    def train():
        loss_value = 1e8
        patience = round(training_iterations/50)
        wait = 0

        for _ in range(training_iterations):
            # Zero backprop gradients
            optimizer.zero_grad()

            # Get output from model
            output = model(x)

            # Calc loss and backprop derivatives
            loss = -mll(output, g)
            loss.backward()
            optimizer.step()

            # Callback
            if loss_value <= loss.item():
                wait += 1
            else:
                wait = 0

            if wait > patience:
                break

            loss_value = loss.item()

            # print(f"Iter {_+1} - Loss: {loss.item()}")
        print(f'Loss: {loss.item()}')
    train()

    # print('Test MAE: \
    # {}'.format(torch.mean(torch.abs(preds.mean - test_y))))
    return model, likelihood


## Initial sampling plan (Latin Hypercube sampling)

In [ ]:
def LHS_sample(num_points, DIM):
    # Number of variables and sampling points
    size = int(num_points)

    # Generate LHS samples
    sampler = qmc.LatinHypercube(d=DIM,
                                 optimization="random-cd")
    lhs_sample = sampler.random(n=size)

    return lhs_sample

## Expected Improvement

In [ ]:
def expected_improvement(x, model):
    """
    Function to calculate the Expected Improvement using Monte Carlo Integration.
    
    Parameters:
        x (array): Individual under evaluation.
        y (array): Valor mínimo obtido até o momento.
    
    Returns:
        array: The value of the Expected Improvement.
    """
    # Get the minimum value obtained so far
    ymin = torch.min(model.train_outputs)
    # ymin = torch.Tensor(ymin)

    # Calculate the prediction value and the variance (Ssqr)
    with torch.no_grad(), use_toeplitz(False), fast_pred_var():
        preds = model(x)
    f = preds.mean
    s = preds.variance

    # Check for any errors that are less than zero (due to numerical error)
    s[s < 0] = 0  # Set negative variances to zero

    # Calculate the RMSE (Root Mean Square Error)
    s = torch.sqrt(s)

    # Calculation of Expected Improvement
    term1 = (ymin - f) * (0.5 + 0.5 * erf((ymin - f) / (s * torch.sqrt( torch.from_numpy(np.array([2])) ))))
    term2 = (1 / torch.sqrt(2 * torch.from_numpy(np.array([np.pi])))) * s * torch.exp(-((ymin - f) ** 2) / (2 * s ** 2))
    
    return -(term1 + term2)

## Objective function

In [ ]:
def obj_fun(x):
    # Entra com x e constrói o modelo
    coordenadas(x)
    barras
    secoes
    materiais
    massa(x)
    
    # Análisa cada caso de carregamento no ANSYS
    P, h = 0, 1e5
    for carregamento in range(n_carregamentos):
        M, N, d = analisa_no_ANSYS(modelo)
        for elemento in range(n_elementos):
            if abs(d[elemento]) >= d_adm:
                P += h*(d[elemento] - d_adm)/d_adm
            
            if abs(N[elemento]) >= N_adm:
                P += h*(N[elemento] - N_adm)/N_adm
        
        P /= n_elementos
    P /= n_carregamentos
    
    # Penaliza (se precisar) o Momento, força axial e deslocamento de cada estaca,
    # para cada caso de carregamento e soma na penalização
    y = massa + P

    if isinstance(y, float):
        return torch.Tensor([y])
    return torch.Tensor(y)

## Define the problem

### Parameters of the optimization

In [ ]:
TRAINING_ITERATIONS = 2000  # epochs for training the GPR
N_INITIAL = 100  # initial number of points
N_INFILL = 400  # number of infill points
BSA_POPSIZE = 25
BSA_EPOCH = 500

### Set variables

In [ ]:
DIM = 18 # dimension of the problem
LOWER_BOUNDS = torch.zeros((DIM))
UPPER_BOUNDS = torch.ones((DIM))

### Generate initial sampling plan using LHS

In [ ]:
x = LHS_sample(N_INITIAL, DIM)
x = torch.from_numpy(x)
x = x.to(torch.float32)

In [ ]:
f = obj_fun(x)
f = f.view(N_INITIAL)

## Efficient Global Optimization

In [ ]:
model, likelihood = train_model(x, f, TRAINING_ITERATIONS)
model.eval()
likelihood.eval()

it = 0
print(f'\nIteration {it}. Best: {torch.min(f)} kg')

Loss: 1.009805323014733e+25

Iteration 0. Best: 4480.72314453125 kg


In [ ]:
while it < N_INFILL:
    it += 1
    
    # Search for the maximum expected improvement
    new_point = bsa(expected_improvement, low=LOWER_BOUNDS, up=UPPER_BOUNDS,
                    popsize=BSA_POPSIZE, epoch=BSA_EPOCH, data=model)
    x_new = torch.from_numpy(new_point.x)
    EI = new_point.y

    # Objective function at the new point
    f_new = obj_fun(x=x_new)
    
    print(f'Iteration {it} of {N_INFILL}')
    if f_new < torch.min(f): print(f'New best: {f_new:.2f} at position {it}')
    
    # Add new values to the initial sampling
    x = torch.cat((x, torch.from_numpy(np.array([np.asarray(x_new)]))), 0)
    x = x.to(torch.float32)
    f = torch.cat((f, f_new), 0)
    
    # Update model
    model, likelihood = train_model(x, f, TRAINING_ITERATIONS)
    model.eval()
    likelihood.eval()
    
    print(f'\nIteration {it}. Best: {torch.min(f)} kg')
    
    # if abs(EI) < TOL_MIN_EI:
    #     print('Optimization finished. Minimum tolerance achieved.')
    #     break

Iteration 1 of 200
Loss: 1.0076856768285132e+25

Iteration 1. Best: 4480.72314453125 kg
Iteration 2 of 200
Loss: 1.0040422142896547e+25

Iteration 2. Best: 4480.72314453125 kg
Iteration 3 of 200
Loss: 1.0017775305781555e+25

Iteration 3. Best: 4480.72314453125 kg
Iteration 4 of 200
Loss: 9.988835823094418e+24

Iteration 4. Best: 4480.72314453125 kg
Iteration 5 of 200
Loss: 1.0117107563853967e+25

Iteration 5. Best: 4480.72314453125 kg
Iteration 6 of 200
Loss: 1.0248187821398738e+25

Iteration 6. Best: 4480.72314453125 kg
Iteration 7 of 200
Loss: 1.0224989887804543e+25

Iteration 7. Best: 4480.72314453125 kg
Iteration 8 of 200
Loss: 1.0181254963448787e+25

Iteration 8. Best: 4480.72314453125 kg
Iteration 9 of 200
Loss: 1.0169700384129617e+25

Iteration 9. Best: 4480.72314453125 kg
Iteration 10 of 200
Loss: 1.0146694988426692e+25

Iteration 10. Best: 4480.72314453125 kg
Iteration 11 of 200
Loss: 1.011815672242316e+25

Iteration 11. Best: 4480.72314453125 kg
Iteration 12 of 200
Loss: 1.00

### Save results

In [ ]:
results = dict()
results["x"] = x
results["f"] = f
results["n_initial"] = N_INITIAL
results["n_infill"] = N_INFILL
results["OFEs"] = N_INITIAL + it

In [ ]:
print(f'f*: {torch.min(f)} kg; x*: {x[torch.argmin(f), :]}')

f*: 4480.72314453125 kg; x*: tensor([0.7097, 0.5206, 0.7261, 0.2552, 0.5334, 0.1589, 0.0682, 0.9707, 0.6417,
        0.7202, 0.3730, 0.8859, 0.2567, 0.7863, 0.8893, 0.6273, 0.8870, 0.6643,
        0.3576, 0.7533, 0.1121, 0.6268, 0.4831, 0.7478, 0.7098, 0.5823, 0.0339,
        0.6778, 0.8060, 0.4855, 0.3113, 0.6491, 0.1007, 0.7651, 0.4384, 0.0853,
        0.0815, 0.0304, 0.7789, 0.2740, 0.6065, 0.0547, 0.7040, 0.7034, 0.0755,
        0.7931, 0.5590, 0.3406, 0.6881, 0.9228, 0.9669, 0.7053])


# BSA

In [ ]:
BSA_POPSIZE = 12  # Ryzen 5 3600x (números múltiplos de 12)
BSA_EPOCH = 200  # depende do tempo computacional
results_BSA = bsa(obj_fun, low=LOWER_BOUNDS, up=UPPER_BOUNDS,
                    popsize=BSA_POPSIZE, epoch=BSA_EPOCH, data=[])
print(f'f*: {results_BSA.y} kg; x*: {results_BSA.x}')


KeyboardInterrupt



In [ ]:
plt.plot(np.arange(
         results_BSA.convergence.shape[0]),
         results_BSA.convergence,
         linewidth=2.5)

plt.xlabel('Iteration')
plt.ylabel('Cost')
# plt.ylim([ymin, ymax])

fig = plt.gcf()
fig.set_size_inches(16, 9)
plt.show()